In [1]:
# 시뮬레이션 라이브러리 불러오기
import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
from autograd import tensor_jacobian_product, grad
import nlopt
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import pickle
import math
import os
import sys

In [2]:
def Material_fit(Material_data_csv=".",
                 eps_inf_min=1.1, eps_inf_max=1.2, eps_inf_linespace=1,
                 wl_min=0.39, wl_max=0.73,
                 num_L=2, num_rep=100):
    # --- 0) 피클 경로 설정 ---
    base, _ = os.path.splitext(Material_data_csv)
    pickle_path = base + "_fit.pkl"

    # --- 1) 피클이 있으면 로드 후 반환 ---
    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as f:
            eps_inf_best, suscepts = pickle.load(f)
        print(f"Loaded fit from pickle: {pickle_path}")
        return eps_inf_best, suscepts

    # --- 2) CSV 로드 및 준비 ---
    data      = np.genfromtxt(Material_data_csv, delimiter=",")
    wl        = data[:, 0]                    # 파장 (μm)
    eps_total = data[:, 1] + 1j*data[:, 2]     # permittivity

    mask     = (wl >= wl_min) & (wl <= wl_max)
    wl_red   = wl[mask]
    freqs_red = 1.0 / wl_red
    eps_total_red = eps_total[mask]
    eps_fit_target_raw = eps_total_red        # eps_inf 뺄 전 원본

    # --- 3) 내부 피팅 함수 ---
    def fit_for_eps_inf(eps_inf):
        eps_fit_target = eps_fit_target_raw - eps_inf
        best_err = np.inf
        best_p   = None
        for _ in range(num_rep):
            p0 = []
            for _ in range(num_L):
                # 초기 파라미터: σ∈[1,100], ω∈[1/λ_max,1/λ_min], γ∈[1e-2,1]
                sigma0 = 10**np.random.uniform(0, 2)
                omega0 = np.random.uniform(1.0/wl_max, 1.0/wl_min)
                gamma0 = 10**np.random.uniform(-2, 0)
                p0.extend([sigma0, omega0, gamma0])
            p_opt, err = lorentzfit(p0, freqs_red, eps_fit_target,
                                    nlopt.LD_MMA, 1e-25, 200000)
            if err < best_err:
                best_err = err
                best_p   = p_opt
        return best_err, best_p

    # --- 4) eps_inf 스윕 및 최적화 ---
    eps_inf_vals = np.linspace(eps_inf_min, eps_inf_max, eps_inf_linespace)
    results = []
    for e_inf in eps_inf_vals:
        err, p_opt = fit_for_eps_inf(e_inf)
        results.append((e_inf, err, p_opt))

    eps_inf_best, err_best, p_opt_best = min(results, key=lambda x: x[1])
    print("Best eps_inf:", eps_inf_best, "error:", err_best)

    # --- 5) susceptibilities 생성 ---
    suscepts = []
    for j in range(num_L):
        ω = p_opt_best[3*j + 1]
        γ = p_opt_best[3*j + 2]
        if abs(ω) < 1e-8:
            σ = p_opt_best[3*j + 0]
            suscepts.append(mp.DrudeSusceptibility(frequency=1.0, gamma=γ, sigma=σ))
        else:
            σ = p_opt_best[3*j + 0] / ω**2
            suscepts.append(mp.LorentzianSusceptibility(frequency=ω, gamma=γ, sigma=σ))

    # --- 6) 결과를 피클로 저장 ---
    with open(pickle_path, "wb") as f:
        pickle.dump((eps_inf_best, suscepts), f)
    print(f"Saved fit to pickle: {pickle_path}")

    return eps_inf_best, suscepts

sio2_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data.csv"
al_data_path   = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/Material data/al_material_data.csv"
andp_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv"

eps_inf_sio2, suscept_sio2 = Material_fit(Material_data_csv=sio2_data_path)
eps_inf_al, suscept_al = Material_fit(Material_data_csv=al_data_path)
eps_inf_andp, suscept_andp = Material_fit(Material_data_csv=andp_data_path)

Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data_fit.pkl
Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/Material data/al_material_data_fit.pkl
Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data_fit.pkl


In [3]:
### ----- Refractive index ----- ###,
sio2 = mp.Medium(epsilon=eps_inf_sio2, E_susceptibilities = suscept_sio2) # SiO2 ,
andp = mp.Medium(epsilon=eps_inf_andp, E_susceptibilities = suscept_andp) # aNDP ,
Al = mp.Medium(epsilon = eps_inf_al, E_susceptibilities = suscept_al)
Air = mp.Medium(index = 1)

In [4]:
resolution = 1000 # 해상도
design_region_width = 0.01 # 디자인 영역 x
design_region_width_y = 0.01 # 디자인 영역 y
design_region_height = 2 # 디자인 영역 높이 z
pml_size = 1.0 # PML 영역 크기

In [5]:
# 시뮬레이션 공간 설정
Sx = design_region_width
Sy = design_region_width_y
Sz = 2 * pml_size + design_region_height + 1 + 5
cell_size = mp.Vector3(0, 0, Sz)

In [ ]:
from pathlib import Path

al_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv")
sio2_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv")
andp_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv")

# CSV 파일 불러오기
andp_data = np.genfromtxt(andp_csv_path, delimiter=",")
wavelengths = andp_data[:,0]
frequencies = 1 / wavelengths
nf = len(frequencies)

In [10]:
minimum_length = 0.05  # minimum length scale (microns)
eta_i = 0.5  # blueprint (or intermediate) design field thresholding point (between 0 and 1)
eta_e = 0.55  # erosion design field thresholding point (between 0 and 1)
eta_d = 1 - eta_e  # dilation design field thresholding point (between 0 and 1)
filter_radius = mpa.get_conic_radius_from_eta_e(minimum_length, eta_e)
design_region_resolution = int(resolution)

pml_layers = [mp.PML(thickness = pml_size, direction = mp.Z)]

In [26]:
wl_ranges = {
    'blue':  (0.38, 0.5),
    'green': (0.5, 0.6),
    'red':   (0.6, 0.72),
}

# 3) mask 생성 후 뽑아내기
width = 0.1  # fwidth 계산용 계수

freqs_color  = {}
fwidths_color = {}

for color, (wl_min, wl_max) in wl_ranges.items():
    mask = (wavelengths >= wl_min) & (wavelengths <= wl_max)
    # mask 가 True 인 인덱스의 frequency 들
    freqs_color[color]   = frequencies[mask]
    # 대응하는 fwidth
    fwidths_color[color] = width * freqs_color[color]

# 결과 확인
print("Red   frequencies:", freqs_color['red'])
print("Red   fwidths:",    fwidths_color['red'], "\n\n")

print("Green frequencies:", freqs_color['green'])
print("Green fwidths:",    fwidths_color['green'], "\n\n")

print("Blue  frequencies:", freqs_color['blue'])
print("Blue  fwidths:",    fwidths_color['blue'])

Red   frequencies: [1.40640251 1.43422432 1.46316911 1.49330625 1.52471099 1.55746501
 1.59165717 1.62738432 1.6647522 ]
Red   fwidths: [0.14064025 0.14342243 0.14631691 0.14933063 0.1524711  0.1557465
 0.15916572 0.16273843 0.16647522] 


Green frequencies: [1.70387649 1.74488704 1.78791726 1.83312344 1.88067494 1.93075912
 1.98358386]
Green fwidths: [0.17038765 0.1744887  0.17879173 0.18331234 0.18806749 0.19307591
 0.19835839] 


Blue  frequencies: [2.03938044 2.09840689 2.16095203 2.22734015 2.29794196 2.3731605
 2.45346994 2.53940522 2.63157895]
Blue  fwidths: [0.20393804 0.20984069 0.2160952  0.22273402 0.2297942  0.23731605
 0.24534699 0.25394052 0.26315789]


{'blue': array([2.03938044, 2.09840689, 2.16095203, 2.22734015, 2.29794196,
        2.3731605 , 2.45346994, 2.53940522, 2.63157895]),
 'green': array([1.70387649, 1.74488704, 1.78791726, 1.83312344, 1.88067494,
        1.93075912, 1.98358386]),
 'red': array([1.40640251, 1.43422432, 1.46316911, 1.49330625, 1.52471099,
        1.55746501, 1.59165717, 1.62738432, 1.6647522 ])}

In [22]:
source_center = [0, 0, Sz / 2 - pml_size] # Source 위치

src_0 = mp.GaussianSource(frequency=fcen_red2, fwidth=fwidth_red2, is_integrated=True)


source = [mp.Source(src_0, component=mp.Ey, center=source_center)]


NameError: name 'fcen_red2' is not defined